### The following code is one of the way to quantify the entanglement of the global system and mixedness of the subsystem. Limited to Pure Two-Qubit system. 

**given a two-qubit pure system. The following approach is folllowed.**
- First we'll get the density matrix $\rho_{AB}$. we can easily verify the properties of density matrix for the calculated $\rho_{AB}$.
- Then we can calculate the linear entropy of the global system. which is the concurrence of the state. Which is calculated by the following equation. $\sqrt{\frac{d}{d-1}  ( 1 - tr(\rho^2) )}$
- The parial trace of the $\rho_{AB}$, will yield $\rho_A$ and $\rho_B$.
- Then calculating the von nuemann entropy for the either of the subsystem, $\rho_A$ or $\rho_B$. by the following: $\sum_i^k-\lambda_ilog_2(\lambda_i)$

In [37]:
import numpy as np
from qiskit.quantum_info import DensityMatrix, partial_trace 

def get_eigen_values(matrix):
    return np.linalg.eigvals(matrix)


def partial_trace(matrix, system):
    # this function is limited to 4x4 matrix. a proper nxn matrix partial trace is yet to be developed
    if system == 0:
        return np.array([
            [matrix[0, 0] + matrix[1, 1], matrix[0, 2] + matrix[1, 3]],
            [matrix[2, 0] + matrix[3, 1], matrix[2, 2] + matrix[3, 3]]
        ])

    if system == 1:
        return np.array([
            [matrix[0, 0] + matrix[2, 2], matrix[0, 1] + matrix[2, 3]],
            [matrix[1, 0] + matrix[3, 2], matrix[1, 1] + matrix[3, 3]]
        ])    
    else:
        raise ValueError("System index must be 0 or 1.")

density_matrix = [[2/3, 0 -(np.sqrt(2)/3)*1j ],
                  [0 + (np.sqrt(2)/3)*1j, 1/3]]

def cal_von_neu_entropy(matrix):
    eigenvalues = get_eigen_values(matrix)
    f_eignevalues = np.round( np.real( eigenvalues ) ,10 )
    entropy = 0
    for eval in f_eignevalues:
        if eval != 0:
            entropy += eval*np.log2(1/eval)

    return entropy

def cal_linear_entropy(matrix):
    matrix = np.array(matrix)
    r = matrix.shape[0]
    rho_square = matrix @ matrix
    normalizing_factor = r/ (r-1)
    linear_ent = 1 - np.trace(rho_square)
    s_prime =  normalizing_factor * np.round(linear_ent, 10)

    return s_prime    


# bell states
bell_phi_plus = np.array([1, 0, 0, 1]) / np.sqrt(2)
bell_phi_minus = np.array([1, 0, 0, -1]) / np.sqrt(2)
bell_psi_plus = np.array([0, 1, 1, 0]) / np.sqrt(2)
bell_psi_minus = np.array([0, 1, -1, 0]) / np.sqrt(2)

def get_the_density_matrix(state):
    density_matrix = np.outer(state, state.conj())
    return density_matrix

def is_density_matrix(rho, tol=1e-8):
    rho = np.array(rho)
    #square matrix check
    if rho.shape[0] != rho.shape[1]:
        # print('1')
        return False
    # Hermitian check
    if not np.allclose(rho, rho.conj().T, atol=tol):
        # print('2')
        
        return False
    # trace check
    if not np.isclose(np.trace(rho), 1, atol=tol):
        # print('3')
        
        return False
    # eigen val check
    evals = get_eigen_values(rho)
    if np.any(evals < -tol):
        # print('4')
        return False
    
    return True



# a method to prepare werner states for the given probability
def prepare_werner_state(p : float):
    identity_4 = np.eye(4)
    return p * np.array(get_the_density_matrix(bell_psi_minus)) + ((1-p)/4) * identity_4


werner_state = prepare_werner_state(p=0)


psi1 = np.array([1, -1, 0, -1 ])/ np.sqrt(3)
psi2 = np.array([1, 1, 1, -1 ])/ 2
psi3 = np.array([1/ np.sqrt(3), np.sqrt(2/3), 0, 0])
# def cal_entanglement(matrix):

rho_AB = get_the_density_matrix(psi2)
if is_density_matrix(rho_AB):

    print("-- density matrix --")
    print(rho_AB)
    print()
    print("-- reduced density matrix (rho_A) --")
    rho_A = np.round(partial_trace(rho_AB, system = 0), 10)
    print(rho_A)
    print("The eigen values are: ", get_eigen_values(rho_A))
    print()
    print("-- reduced density matrix (rho_B) --")
    rho_B = np.round(partial_trace(rho_AB, system = 1), 10)
    print(rho_B)
    print("The eigen values are: ",get_eigen_values(rho_B))
    print()

    von_neumann_entropy = cal_von_neu_entropy(rho_A)
    linear_entropy = cal_linear_entropy(rho_A)

    print("the entanglement of the global system is ", von_neumann_entropy)
    print()

    print("the concurrence of the global system is", linear_entropy)
    print()
    print("---if the measure reaches 1, the system is maximally entangled---"
    )

else:
    print("invalid density matrix")





-- density matrix --
[[ 0.25  0.25  0.25 -0.25]
 [ 0.25  0.25  0.25 -0.25]
 [ 0.25  0.25  0.25 -0.25]
 [-0.25 -0.25 -0.25  0.25]]

-- reduced density matrix (rho_A) --
[[0.5 0. ]
 [0.  0.5]]
The eigen values are:  [0.5 0.5]

-- reduced density matrix (rho_B) --
[[0.5 0. ]
 [0.  0.5]]
The eigen values are:  [0.5 0.5]

the entanglement of the global system is  1.0

the concurrence of the global system is 1.0

---if the measure reaches 1, the system is maximally entangled---


In [ ]:
# entanglement of formation and concurrence
from qiskit.quantum_info import Pauli

pauliYY = np.real(Pauli('YY').to_matrix())

def cal_concurrence(rho):
    rho_star = rho.conj()
    rho_tilde = pauliYY @ rho_star @ pauliYY
    r_matrix = rho @ rho_tilde

    eigenvalues = np.sqrt( np.round( get_eigen_values(r_matrix) ,3) ).tolist()
    eigenvalues.sort(reverse = True)
    # sorted_values = eigenvalues.tolist()  

    final = eigenvalues[0] - np.sum(eigenvalues[1:4])

    return max(0, final)

matrix = np.array([[1, 0, 0, 0],[0, 0, 0, 0],[0, 0, 0, 0],[0,0,0, 1]]) / 2
# print(matrix)
concurrence = cal_concurrence(matrix)

def cal_entanglement(c):
    
    h = lambda x: -x* np.log2(x) - (1-x) * np.log2(1-x)
    if c == 0:
        return 0
    else: 
        ent = h( (1 + np.sqrt(1 - c ** 2))/ 2 )
    return ent
print(f"the concurrence of the system is {concurrence}")
print()
entanglement = cal_entanglement(concurrence)
print(f"the entanglement of the system is {entanglement}")

the concurrence of the system is 0

the entanglement of the system is 0
